In [1]:
import os

folders = ["raw", "processed", "output"]
for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("Folders created!")


Folders created!


In [2]:
import pandas as pd

df = pd.read_csv("retail_sales_dataset.csv")


In [3]:
print("Initial Shape:", df.shape)
df.head()

Initial Shape: (1000, 9)


,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [4]:
print("\nMissing Values:\n", df.isnull().sum())
print("\nDuplicate Rows:", df.duplicated().sum())



Missing Values:
 Transaction ID      0
Date                0
Customer ID         0
Gender              0
Age                 0
Product Category    0
Quantity            0
Price per Unit      0
Total Amount        0
dtype: int64

Duplicate Rows: 0


In [5]:
df = df.drop_duplicates()


In [6]:
df = df.dropna()


In [7]:
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

df.head()


,transaction_id,date,customer_id,gender,age,product_category,quantity,price_per_unit,total_amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100


In [8]:
df["date"] = pd.to_datetime(df["date"])
df["price_per_unit"] = df["price_per_unit"].astype(float)
df["total_amount"] = df["total_amount"].astype(float)
df["quantity"] = df["quantity"].astype(int)


In [9]:
df["calculated_revenue"] = df["quantity"] * df["price_per_unit"]


In [10]:
df["high_value_flag"] = df["total_amount"].apply(lambda x: 1 if x > 500 else 0)


In [11]:
def age_group(age):
    if age < 25:
        return "Young"
    elif age < 40:
        return "Adult"
    else:
        return "Senior"

df["age_group"] = df["age"].apply(age_group)


In [12]:
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month


In [13]:
customers = df[["customer_id", "gender", "age", "age_group"]].drop_duplicates()


In [14]:
products = df[["product_category", "price_per_unit"]].drop_duplicates()


In [18]:
transactions = df[[
    "transaction_id",
    "date",
    "customer_id",
    "product_category",
    "quantity",
    "price_per_unit",
    "total_amount",
    "high_value_flag"
]]


In [16]:
customers.to_csv("processed/customers.csv", index=False)
products.to_csv("processed/products.csv", index=False)
transactions.to_csv("processed/transactions.csv", index=False)

df.to_csv("processed/processed_data.csv", index=False)

print("Processed files saved!")


Processed files saved!


In [17]:
import sqlite3

conn = sqlite3.connect("output/database.sqlite")

customers.to_sql("customers", conn, if_exists="replace", index=False)
products.to_sql("products", conn, if_exists="replace", index=False)
transactions.to_sql("transactions", conn, if_exists="replace", index=False)

conn.close()

print("Database created successfully!")


Database created successfully!


In [19]:
print("Final Shape:", df.shape)
print("Customers:", customers.shape)
print("Products:", products.shape)
print("Transactions:", transactions.shape)


Final Shape: (1000, 14)
Customers: (1000, 4)
Products: (15, 2)
Transactions: (1000, 8)
